In [73]:
import pandas as pd

In [74]:
match_stats_df = pd.read_csv('../data/Premier_League_all_match_stats_2015-2023.csv')

In [75]:
match_stats_df.head()

,Season,Match_Date,Home_Team_Name,Away_Team_Name,Home_Goals,Away_Goals,Home_shots_on_goal,Home_shots_off_goal,Home_total_shots,Home_blocked_shots,...,Away_total_passes,Away_passes_accurate,Away_passes_%,Home_offsides,Away_red_cards,Home_red_cards,Home_fouls,Away_fouls,Home_expected_goals,Away_expected_goals
0,2015,1032016,Aston Villa,Everton,1,3,5.0,4.0,14.0,5.0,...,537,442,82%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1032016,AFC Bournemouth,Southampton,2,0,4.0,2.0,9.0,3.0,...,424,327,77%,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1032016,Leicester City,West Bromwich Albion,2,2,4.0,11.0,22.0,7.0,...,273,168,62%,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1032016,Norwich City,Chelsea,1,2,2.0,7.0,13.0,4.0,...,520,427,82%,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1032016,Sunderland,Crystal Palace,2,2,6.0,7.0,17.0,4.0,...,293,192,66%,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
fifa_ratings_df = pd.read_csv('../data/merge-csv.com__655bb56e2f110.csv')

In [77]:
fifa_ratings_df.head()

,Game Version,Update Date,Team Avatar,Team Name,ID,Overall,Attack,Midfield,Defence,Transfer budget,...,Shooting,Aggression,Pressure,Team width,Defender line,Domestic prestige,International prestige,Players,Starting XI average age,Whole team average age
0,FC 24,Nov 15 2023,https://cdn.sofifa.net/meta/team/9/60.png,Manchester City,10,85,87,86,83,€0,...,Little,Contain,Deep,Narrow,Cover,10,10,29,26.18,25.14
1,FC 24,Nov 15 2023,https://cdn.sofifa.net/meta/team/18645/60.png,England,1318,85,86,83,83,€0,...,Little,Contain,Deep,Narrow,Cover,10,8,26,26.73,26.50
2,FC 24,Nov 15 2023,https://cdn.sofifa.net/meta/team/19/60.png,Arsenal,1,83,82,84,82,€0,...,Little,Contain,Deep,Narrow,Cover,8,7,32,24.00,23.44
3,FC 24,Nov 15 2023,https://cdn.sofifa.net/meta/team/8/60.png,Liverpool,9,83,84,81,84,€0,...,Little,Contain,Deep,Narrow,Cover,9,9,33,26.09,23.94
4,FC 24,Nov 15 2023,https://cdn.sofifa.net/meta/team/14/60.png,Manchester United,11,82,80,83,78,€0,...,Little,Contain,Deep,Narrow,Cover,9,8,33,26.73,25.03


In [78]:
selected_columns = ['Overall', 'Attack', 'Midfield', 'Defence', 'Transfer budget', 'Club worth',
                     'Speed', 'Dribbling', 'Passing', 'Positioning', 'Crossing', 'Shooting',
                     'Aggression', 'Pressure', 'Team width', 'Defender line', 'Domestic prestige',
                     'International prestige', 'Players', 'Starting XI average age', 'Whole team average age']

In [79]:
merged_df = pd.DataFrame()

for idx, row in match_stats_df.iterrows():
    print(f"Processing match {idx + 1}/{len(match_stats_df)}")

    print(f"{row['Home_Team_Name']} ({row['Home_Goals']}) x ({row['Away_Goals']}) {row['Away_Team_Name']}\n")

    year = int(str(row['Season']))

    home_team_rating = fifa_ratings_df.loc[
        (fifa_ratings_df['Team Name'] == row['Home_Team_Name']) &
        (fifa_ratings_df['Update Date'].str.contains(str(year))),
        selected_columns
    ].head(1)

    if home_team_rating.empty:
        home_team_rating = pd.Series(index=selected_columns)
    else:
        home_team_rating = home_team_rating.add_prefix('Home_')

    away_team_rating = fifa_ratings_df.loc[
        (fifa_ratings_df['Team Name'] == row['Away_Team_Name']) &
        (fifa_ratings_df['Update Date'].str.contains(str(year))),
        selected_columns
    ].head(1)

    if away_team_rating.empty:
        away_team_rating = pd.Series(index=selected_columns)
    else:
        away_team_rating = away_team_rating.add_prefix('Away_')

    merged_row = dict(row)
    merged_row.update(home_team_rating.to_dict())
    merged_row.update(away_team_rating.to_dict())

    for key in merged_row.keys():
        if isinstance(merged_row[key], dict):
            merged_row[key] = next(iter(merged_row[key].values()), None)

    merged_df = pd.concat([merged_df, pd.DataFrame(merged_row, index=[0])], axis=0, ignore_index=True)

Processing match 1/2888
Aston Villa (1) x (3) Everton

   Season  Match_Date Home_Team_Name Away_Team_Name  Home_Goals  Away_Goals  \
0    2015     1032016    Aston Villa        Everton           1           3   

   Home_shots_on_goal  Home_shots_off_goal  Home_total_shots  \
0                 5.0                  4.0              14.0   

   Home_blocked_shots  ...  Away_Shooting  Away_Aggression  Away_Pressure  \
0                 5.0  ...         Normal            Press         Medium   

  Away_Team width  Away_Defender line  Away_Domestic prestige  \
0          Normal               Cover                      14   

   Away_International prestige  Away_Players Away_Starting XI average age  \
0                           16            33                        26.36   

   Away_Whole team average age  
0                        24.97  

[1 rows x 82 columns]
Processing match 2/2888
AFC Bournemouth (2) x (0) Southampton

   Season  Match_Date   Home_Team_Name Away_Team_Name  Home_Goal

KeyboardInterrupt: 

In [ ]:
merged_df.to_csv('../data/all_games_stats_with_team_ratings.csv', index=False)